In [1]:
Coord = tuple[int, int]
coords: list[Coord]
with open('18.txt') as f:
    coords = [
        tuple(map(int, line.split(',')))
        for line in f.readlines()]

START: Coord = (0, 0)
END: Coord = (70, 70)
STEPS = 1024

In [2]:
def after(coords: list[Coord], steps: int, size: Coord):
    width, height = size
    grid = [['.' for _ in range(width+1)] for _ in range(height+1)]
    for x, y in coords[:steps]:
        grid[y][x] = '#'
    return grid

def display(grid: list[list[str]], wait=0):
    from IPython.display import clear_output
    from time import sleep

    print('\n'.join(''.join(line) for line in grid))
    if wait:
        clear_output(True)
        sleep(wait)

# Part 1: A* algorithm

In [3]:
from queue import PriorityQueue

DIRECTIONS = [
    (-1, 0),
    (+1, 0),
    (0, -1),
    (0, +1)
]

def a_star(grid: list[list[str]], start: Coord, end: Coord):
    UNSEEN_COST = len(grid) * len(grid[0]) + 1
    frontier = PriorityQueue[Coord]()
    frontier.put(start, 0)
    costs: dict[Coord, int] = {}
    costs[start] = 0
    prevs: dict[Coord, Coord] = {}
    prevs[start] = None

    ex, ey = end

    while not frontier.empty():
        x, y = frontier.get()
        if (x, y) == end:
            break

        for dx, dy in DIRECTIONS:
            sx, sy = x+dx, y+dy
            if not 0 <= sy < len(grid):
                continue
            if not 0 <= sx < len(grid[sy]):
                continue
            if grid[sy][sx] == '#':
                continue
            cost = costs[x, y] + 1
            if cost < costs.get((sx, sy), UNSEEN_COST):
                costs[sx, sy] = cost
                prevs[sx, sy] = x, y
                estimate = cost + abs(ex-sx) + abs(ey-sy)
                frontier.put((sx, sy), estimate)
    
    return costs, prevs

grid = after(coords, STEPS, END)
costs, prevs = a_star(grid, START, END)

def get_path(prevs: list[Coord], end: Coord):
    coord = end
    while coord in prevs:
        yield coord
        coord = prevs[coord]

def mark_path(grid: list[list[str]], prevs: list[Coord], end: Coord):
    grid = [list(line) for line in grid]
    for x, y in get_path(prevs, end):
        grid[y][x] = 'O'
    return grid

display(mark_path(grid, prevs, END))
print()
print(costs[END])

O............#OOO#OOO....#.#.......#.#OOO#.............................
O###########.#O#O#O#O.##.#.#...###.#.#O#O#.#...........................
OOOOO#.......#O#OOO#O#...#.....#.....#O#O#.#...........................
.###O###.....#O#####O#.#######.#.#####O#O#....#........................
...#OOO#.#...#OOO#.#O#.........#...#OOO#O#.#...........................
##.###O#.#.#####O#.#O#####.####.##.#O##.O#.#...........................
.#...#O#.....#OOO#.#OOOOO#.#OOOOO#.#O..#O#.#...........................
.###.#O###.###O###.##.##O#.#O###O.##O###O#...#.........................
...#.#OOO..#OOO#.....#OOO#.#OOO#O#OOO#OOO#.#...#.......................
..#..###O###O###.###.#O###.###O#O#O##.O#.#..#..........................
...#...#OOOOO#...#...#OOO#.#OOO#O#O#.#O#.#.............................
....##.#######.###.##.##O###O###O#O#.#O#.#.#...........................
.....#.#.......#.#.....#OOO#O#.#O#O..#O#.#.............................
.....#.###.####..#####.###O#O#.#O#O###O#.#.#.#..................

# Part 2: Endless Corruption

In [4]:
width, height = END
grid = [['.' for _ in range(width+1)] for _ in range(height+1)]
SAFE_START = 1024
path = None

N_paths = 0

for N, (x, y) in enumerate(coords):
    grid[y][x] = '#'
    generate_new_paths = (x, y) in path if path else True
    if N > SAFE_START and generate_new_paths:
        costs, prevs = a_star(grid, START, END)
        path = set(get_path(prevs, END))
        N_paths += 1
        
        if END not in costs:
            break
        max_cost = costs[END]
        # if N % 100 == 0:
        #     display(mark_path(grid, prevs, END), 1)

max_steps = N
print(f'After {N} steps and {N_paths} path calculations,')
print(f'cell {x},{y} finally blocked the path')
print(f'(one step before that, the max cost was {max_cost})')

After 2877 steps and 69 path calculations,
cell 39,40 finally blocked the path
(one step before that, the max cost was 960)


# Bonus Round! Animating Part 2

In [5]:
from pathlib import Path
from PIL import Image
from matplotlib import colormaps
from tqdm import tqdm

THEME = colormaps['viridis']

ANIM_DIR = Path('./frames')
ANIM_DIR.mkdir(exist_ok=True)

width, height = END
grid = [['.' for _ in range(width+1)] for _ in range(height+1)]

PIXEL_SIZE = 10
image = Image.new('RGB', ((width+1) * PIXEL_SIZE, (height+1) * PIXEL_SIZE))

for N, (x, y) in tqdm(enumerate(coords), total=max_steps):
    grid[y][x] = '#'
    image.paste((256, 256, 256), (0, 0, image.width, image.height))

    def pixel(coord: Coord, col):
        x, y = coord
        box = (x*PIXEL_SIZE, y*PIXEL_SIZE, (x+1)*PIXEL_SIZE, (y+1)*PIXEL_SIZE)
        image.paste(col, box)

    costs, prevs = a_star(grid, START, END)
    if END not in costs:
        break

    for (x, y), cost in costs.items():
        r, g, b, _ = THEME(cost / max_cost)
        R, G, B = int(r*256), int(g*256), int(b*256)
        pixel((x, y), (R, G, B))
    
    coord = END
    while coord in prevs:
        x, y = coord
        pixel((x, y), (0, 0, 0))
        coord = prevs[coord]
    
    image.save(ANIM_DIR / f'{N:04}.png')

from os import chdir, system

chdir(ANIM_DIR)
system("ffmpeg -r 60 -f image2 -s 1920x1080 -i './%04d.png' -vcodec libx264 -crf 25 ../18.mp4")

100%|██████████| 2877/2877 [03:00<00:00, 15.95it/s]
ffmpeg version 7.1 Copyright (c) 2000-2024 the FFmpeg developers
  built with Apple clang version 16.0.0 (clang-1600.0.26.3)
  configuration: --prefix=/opt/homebrew/Cellar/ffmpeg/7.1 --enable-shared --enable-pthreads --enable-version3 --cc=clang --host-cflags= --host-ldflags='-Wl,-ld_classic' --enable-ffplay --enable-gnutls --enable-gpl --enable-libaom --enable-libaribb24 --enable-libbluray --enable-libdav1d --enable-libharfbuzz --enable-libjxl --enable-libmp3lame --enable-libopus --enable-librav1e --enable-librist --enable-librubberband --enable-libsnappy --enable-libsrt --enable-libssh --enable-libsvtav1 --enable-libtesseract --enable-libtheora --enable-libvidstab --enable-libvmaf --enable-libvorbis --enable-libvpx --enable-libwebp --enable-libx264 --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lzma --enable-libfontconfig --enable-libfreetype --enable-frei0r --enable-libass --enable-libopencore-amrnb --enable-libopencor

0